In [2]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv1D, Dense, Dropout, BatchNormalization, LSTM
from tensorflow.keras.optimizers import Adam
from sklearn.metrics import classification_report
from sklearn.preprocessing import LabelBinarizer
import matplotlib.pyplot as plt

# Ensure eager execution is enabled (it's the default in TF2)
tf.config.run_functions_eagerly(True)

# Load the preprocessed data
X_train = np.loadtxt("../../../../Datasets/CAPP Dataset/Subject Independent SW/X_train.txt")
y_train = np.loadtxt("../../../../Datasets/CAPP Dataset/Subject Independent SW/y_train.txt")
X_test = np.loadtxt("../../../../Datasets/CAPP Dataset/Subject Independent SW/X_test.txt")
y_test = np.loadtxt("../../../../Datasets/CAPP Dataset/Subject Independent SW/y_test.txt")

# Reshape the input data for Conv1D layers
n_timesteps = 100
n_features = 9
X_train = X_train.reshape(X_train.shape[0], n_timesteps, n_features)
X_test = X_test.reshape(X_test.shape[0], n_timesteps, n_features)

# One-hot encode the labels
lb = LabelBinarizer()
y_train = lb.fit_transform(y_train)
y_test = lb.transform(y_test)

def create_model(n_timesteps, n_features, n_outputs):
    model = Sequential([
        Conv1D(filters=64, kernel_size=3, activation="relu", input_shape=(n_timesteps, n_features)),
        BatchNormalization(),
        Conv1D(filters=64, kernel_size=3, activation="relu"),
        BatchNormalization(),
        Conv1D(filters=64, kernel_size=3, activation="relu"),
        BatchNormalization(),
        LSTM(100, return_sequences=True),
        Dropout(0.5),
        LSTM(100),
        Dropout(0.5),
        Dense(100, activation="relu"),
        BatchNormalization(),
        Dense(n_outputs, activation="softmax")
    ])
    return model

n_outputs = y_train.shape[1]
model = create_model(n_timesteps, n_features, n_outputs)
model.compile(optimizer=Adam(learning_rate=0.001), loss="categorical_crossentropy", metrics=["accuracy"])

history = model.fit(X_train, y_train, epochs=50, batch_size=32, validation_split=0.2, verbose=1)

# Evaluate the model
y_pred = model.predict(X_test)
y_pred_classes = np.argmax(y_pred, axis=1)
y_true_classes = np.argmax(y_test, axis=1)

# Print classification report
print(classification_report(y_true_classes, y_pred_classes, digits=5))

# Convert the model to TensorFlow Lite format
converter = tf.lite.TFLiteConverter.from_keras_model(model)
tflite_model = converter.convert()

# Save the TFLite model
with open('model.tflite', 'wb') as f:
    f.write(tflite_model)

print("TensorFlow Lite model has been saved as 'model.tflite'")

# If you still want to generate the C header file, you can use the everywhereml library
# Note that this might not be compatible with the TFLite model, so you may need to choose one or the other
from everywhereml.code_generators.tensorflow import convert_model

c_header = convert_model(model, X_test, y_test, model_name='deep_conv_lstm')
print(c_header)

with open('GeneratedHeaderFiles/deep_conv_lstm.h', 'w') as file:
    file.write(c_header)

print("C header file has been saved as 'GeneratedHeaderFiles/deep_conv_lstm.h'")


/home/denuvo-drm/miniconda3/envs/CompositeADLRecognition/lib/python3.10/site-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
/home/denuvo-drm/miniconda3/envs/CompositeADLRecognition/lib/python3.10/site-packages/tensorflow/python/data/ops/structured_function.py:258: UserWarning: Even though the `tf.config.experimental_run_functions_eagerly` option is set, this option does not apply to tf.data functions. To force eager execution of tf.data functions, please use `tf.data.experimental.enable_debug_mode()`.
  warnings.warn(


Epoch 1/50


RuntimeError: `tf.data.Dataset` only supports Python-style iteration in eager mode or within tf.function.